In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from category_encoders import TargetEncoder, OneHotEncoder, BinaryEncoder, CountEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import random

In [ ]:
df = pd.read_csv('https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1')
df.head(10)

Patrząc na kilka pierwszych rekordów z wczytanej ramki danych, możemy zauważyć, że wielkość liter w nazwach lokacji jest różna. Ujednolićmy więc dane nazwy zmieniając ich pisownię na wielkie litery.

In [ ]:
df[["it_location"]] = df[["it_location"]].apply(lambda x: x.str.upper())

## 1. Kodowanie zmiennych kategorycznych

### Target encoding

In [ ]:
df_target_encoding = df.copy()
df_target_encoding[["it_location"]].describe()

In [ ]:
encoder = TargetEncoder()
df_target_encoding["it_location_target_encoded"] = encoder.fit_transform(
    df_target_encoding["it_location"], df_target_encoding["price"])
df_target_encoding[["it_location_target_encoded"]].describe()

In [ ]:
len(df_target_encoding[["it_location_target_encoded"]].value_counts())

Możemy zauważyć, że zmienna it_location przyjmuje aż 7903 róznych wartości. Przy zastosowaniu one hot encodingu wynikowa ramka danych miałaby 7903 nowych kolumn, co zdecydowanie nie byłoby dobrym wyjściem. Możemy zauważyć, że po zastosowaniu target encodingu liczba unikatowych wartości zmniejszyła się do 5140 (w wynikowej, zakodowanej kolumnie).

Wartości w nowo powstałej kolumnie są wyliczoną średnią wartością zmiennej docelowej cechy dla każdej z kategorii.

Target encoding jednak może prowadzić do przeuczenia modelu (jest powiązany ze zmienną objaśnialną). Wprowadza także możliwość porównywania zakodowanych w ten sposób kategorii, co w naszym przypadku wydaje się nie mieć dużego sensu.

#### Kodowanie main_category

In [ ]:
df[['main_category']].describe()

##### One Hot Encoder

In [ ]:
oh_encoder = OneHotEncoder(use_cat_names=True).fit_transform(df["main_category"])
oh_encoder.head()

Powstało 27 nowych kolumn, każda dla unikalnej wartości zmiennej main_category. 

##### Count Encoder

In [ ]:
count_encoder = CountEncoder().fit_transform(df["main_category"])
count_encoder.head()

Kodowanie count encoder tworzy nam jedną kolumnę, gdzie dla każdej obserwacji przypisana jest liczba produktów z odpowiadającej kategorii. 

##### Binary Encoder

In [ ]:
binary_encoder = BinaryEncoder().fit_transform(df["main_category"])
binary_encoder.head()

Kodowanie binarne jest bardzo podobne do One Hote'a, ale przechowuje kategorie w postaci binarnych bitstringów.

## 2. Uzupełnianie braków

In [ ]:
df2 = df[['price','it_seller_rating','it_quantity']].copy()
df2 = df2.sample(frac=0.1)
df2.head()

In [ ]:
rmse_errors = []

for i in range(10):
    df2_1 = df2.copy()
    remove_index = df2_1.sample(frac=0.1).index
    df2_1.loc[remove_index, 'it_seller_rating'] = None
    df2_1 = pd.DataFrame(KNNImputer(weights='uniform', n_neighbors=5).fit_transform(df2_1), columns=df2_1.columns)
    error = np.sqrt(mean_squared_error(df2["it_seller_rating"], df2_1["it_seller_rating"]))
    rmse_errors.append(error)
    print(error)
    

In [ ]:
np.std(rmse_errors)

In [ ]:
rmse_errors1 = []
rmse_errors2 = []

for i in range(10):
    df2_2 = df2.copy()
    remove_index1 = df2_2.sample(frac=0.1).index
    remove_index2 = df2_2.sample(frac=0.1).index
    
    df2_2.loc[remove_index1, 'it_seller_rating'] = None
    df2_2.loc[remove_index2, 'it_quantity'] = None
    
    df2_2 = pd.DataFrame(KNNImputer(weights='uniform', n_neighbors=5).fit_transform(df2_2), columns=df2_2.columns)
    error1 = np.sqrt(mean_squared_error(df2["it_seller_rating"], df2_2["it_seller_rating"]))
    error2 = np.sqrt(mean_squared_error(df2["it_quantity"], df2_2["it_quantity"]))
    
    rmse_errors1.append(error1)
    rmse_errors2.append(error2)
    print("RMSE it_seller_rating - ", error1)
    print("RMSE it_quantity - ", error2)

In [ ]:
print("RMSE it_seller_rating std - ",np.std(rmse_errors1))
print("RMSE it_quantity std - ",np.std(rmse_errors2))

In [ ]:
plt.plot(range(1,11), rmse_errors, range(1,11), rmse_errors1)
plt.legend(['Usunięcie it_seller_rating', 'Usunięcie obu kolumn'])

In [ ]:
plotdf = pd.DataFrame({"1": rmse_errors, "2":rmse_errors1})
fig, ax = plt.subplots(figsize=(12,8))
sns.boxplot(data=plotdf)

Na wykresach możemy zaobserwować, że przy usunięciu danych z dwóch kolumn imputacja będzie obarczona większym błędem, choć nadal losowość w doborze usuwanych danych może powodować pewne odstępstwa.   